In [1]:
from quspin.operators import hamiltonian,exp_op,quantum_operator # operators expop for exponential
from quspin.basis import spinful_fermion_basis_1d  # Hilbert space This class defines the basis states for a one-dimensional system of spinful fermions. 
# Each state in this basis specifies which lattice sites are occupied by fermions and what their spins are.
from quspin.tools.measurements import obs_vs_time  # calculating dynamics   allows for the calculation of observables (measurable quantities like particle
#number, magnetization, etc.) as they evolve over time. It’s used to analyze how the quantum system changes dynamically.
import numpy as np
from numpy.random import uniform,choice # tools for doing random sampling uniform- generates random numbers uniformly distributed potential over a specified range.
#It is often used to introduce randomness in parameters like disorder in a quantum system.
# choice-This function randomly selects elements from a given list or array. It can be used to randomly sample from a set of possible values, such as 
# selecting random sites in a lattice. which we use to estimate the uncertainties of the disorder averages using a bootstrap re-sampling procedure
from time import time # tools for calculating computation time

In [2]:
#setting  parameters for simulation
# simulation parameters 
n_real=100 #number of realizations realization refers to a single instance of the system with a specific disorder configuration. Running multiple 
#realizations allows for averaging over different disorder configurations, which is important for studying systems with randomness.

n_boot=100 # number of bootstrap samples to calculate error  used to calculate the error in measurements. Bootstrapping is a statistical method where you
#repeatedly sample with replacement from a dataset to estimate the distribution of a statistic (e.g., the mean). Here, it's used to assess the reliability 
#of the results by calculating the error.

# physical parameters 

L=8 #system size no of sites in 1d lattice
N=L//2 # number of particles // means integer division
N_up=N//2+N%2 # number of fermions with spin up % for reminder
N_down= N//2 # number of fermions with spin down 
w_list = [1.0,4.0,10.0] # disorder strength w that will be used in the simulation. Disorder strength  determines how much randomness is present in the 
# on-site potential of the lattice. The simulation will be run for each of these disorder strengths to study their effects on the system's behavior.

J=1.0 # hopping strength amplitude for a particle to move (or "hop") from one site to an adjacent site.
U=5.0 # interaction strength  interaction strength 𝑈, which represents the energy cost for two fermions with opposite spins to occupy the same site
#range in terms to evolve system
start,stop,num=0.0,35.0,101 #define the start time, stop time, and the number of time points, respectively, for the time evolution of the system.
t=np.linspace(start ,stop,num=num,endpoint=True) 
# creates an array t of num evenly spaced time points between start and stop. The endpoint=True ensures that the stop value 35.0 is included in the array.
# np.linspace(): A NumPy function that generates an array of evenly spaced values over a specified range. Here, it generates 101 time points between 0.0 and 35.0, inclusive

In [3]:
# craete the basis 
# build spinful fermion basis
basis=spinful_fermion_basis_1d(L,Nf=(N_up,N_down)) # Nf=(N_up, N_down) tells the spinful_fermion_basis_1d class that the basis should be constructed for
# a system where there are N_up fermions with spin-up and N_down fermions with spin-down, distributed among the L lattice sites.

#craete model
#define site-coupling lists

hop_right =[[-J,i,i+1]for i in range(L-1)]#hoppingtotherightOBC -J coressponds to hoping right
hop_left=[[J,i,i+1]for i in range(L-1)]#hoppingtotheleftOBC  J hoping to left

int_list=[[U,i,i]for i in range(L)]#onsiteinteraction  interaction between fermions on the same site i. The term [U, i, i] corresponds to an interaction
#strength U between fermions at site i. This is typically a repulsive interaction (Coulomb repulsion) if U is positive.



#site-coupling list to create the sublattice imbalance observable
sublat_list=[[(-1.0)**i/N,i]for i in range( 0,L)]


  #createstaticlists
operator_list_0 =[
["+-|",hop_left],#uphopleft  hopping of spin-up fermions to the left. "+-|" indicates an annihilation (-) of a fermion at one site and a creation (+) at
    #an adjacent site, thus representing a hop to the left
["-+|",hop_right], # uphopright  hopping of spin-up fermions to the right. "|-+" indicates a creation (+) of a fermion at one site and an annihilation (-) 
    #at an adjacent site, thus representing a hop to the right.
["|+-",hop_left],#downhopleft spin down fermion to left
["|-+",hop_right], # downhopright spin down fermion to right
["n|n",int_list],#on site interaction  on-site interaction between spin-up and spin-down fermions on the same site. "n|n" indicates the number operator 
    #n for both spins, which counts the number of particles at a site.
]
imbalance_list=[["n|",sublat_list],["|n",sublat_list]] # used to measure the sublattice imbalance observable
# ["n|", sublat_list]: Measures the imbalance for spin-up fermions. "n|" indicates the number operator n for spin-up fermions.
# ["|n", sublat_list]: Measures the imbalance for spin-down fermions. "|n" indicates the number operator n for spin-down fermions.

In [4]:
#create operator dictionary for quantum_operator class
#add key for Hubbard hamiltonian

operator_dict=dict(H0=operator_list_0) # operator dictionary 
#add keys for local potential in each site
for i in range(L):
    operator_dict["n"+str(i)]=[["n|",[[1.0,i]]],["|n",[[1.0,i]]]]
#add to dictioanry keys h 0,h1,h2,...,hL with local potential operator
  # "H0": The key for the Fermi-Hubbard Hamiltonian."n0", "n1", "n2", ..., "nL-1": Keys for the local potential operators at each site of the lattice.


In [5]:
 ######settingupoperators
#setuphamiltoniandictionaryandobservable(imbalanceI)
no_checks =dict(check_pcon=False,check_symm=False,check_herm=False)
H_dict=quantum_operator(operator_dict,basis=basis,**no_checks)
I=hamiltonian(imbalance_list,[],basis=basis,**no_checks)
#strings whichrepresenttheinitialstate
s_up="".join("1000" for i in range(N_up))
s_down="".join("0010" for i in range(N_down))
 #basis.indexacceptsstringsandreturnstheindex
 #which correspondstothatstateinthebasislist
i_0 =basis.index(s_up,s_down) #findindexofproductstate
psi_0 =np.zeros(basis.Ns)#allocatespaceforstate
psi_0[i_0]=1.0 #setMBstate tobethegivenproductstate
print("H-spacesize:{:d},initialstate:|{:s}>(x)|{:s}>".format(basis.Ns,s_up,s_down))

#definefunctiontododynamicsfordifferentdisorderrealizations.
def real(H_dict, I, psi_0, w, t, i):
    ti = time() 
    params_dict=dict(H0=1.0)
    for j in range(L):
     params_dict["n"+str(j)]=uniform(-w,w)
     H = H_dict.tohamiltonian(params_dict)
     U = exp_op(H,a=-1j,start=t.min(),stop=t.max(),num=len(t),iterate=True)
     psi_t = U.dot(psi_0) # get generator psi_t for time evolved state
     t = U.grid # extract time grid stored in U, and defined in exp_op
     obs_t = obs_vs_time(psi_t,t,dict(I=I))
     t = U.grid # extract time grid stored in U, and defined in exp_op
     obs_t = obs_vs_time(psi_t,t,dict(I=I))
# print reporting the computation time for realization
     print("realization {}/{} completed in {:.2f} s".format(i+1,n_real,time()-ti))
 # return observable values
     return obs_t["I"]
        
     for w in w_list:
            I_data = np.vstack([real(H_dict,I,psi_0,w,t,i) for i in range(n_real)])
            I_avg = I_data.mean(axis=0) # get mean value of I for all time points
 # generate bootstrap samples
            bootstrap_gen = (I_data[choice(n_real,size=n_real)].mean(axis= 0) for i in range(
 n_boot))
 # generate the fluctuations about the mean of I
            sq_fluc_gen = ((bootstrap-I_avg)**2 for bootstrap in bootstrap_gen)
            I_error = np.sqrt(sum(sq_fluc_gen)/n_boot)
    

 
 
         # use exp_op to get the evolution operator
 
#bodyoffunctiongoesbelow
#starttimingfunctionfordurationofreachrealisation

#createaparameterlistwhichspecifiestheonsitepotentialwithdisorder



H-spacesize:784,initialstate:|10001000>(x)|00100010>
